# Batch processing with Spark and Cassandra

### Project setup: Import Spark and Cassandra Libraries, connect to clusters etc

In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import StorageLevel
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *

Now connect to the cassandra and spark clusters

In [2]:
from cassandra.cluster import Cluster
cluster = Cluster()

conf = SparkConf().setAppName("Smart Meter Watchdog")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

## Common Functions used for both high frequency and low frequency data

In [3]:
import time

def ts2date(curTime):
    return time.strftime("%D", time.localtime(int(curTime)))

ts2date('1306006763') ## Should print "05/21/11"

'05/21/11'

In [4]:
def expandRec(rec):
    result = []
    for row in rec.readings:
        result.append((rec.houseId, ts2date(rec.timestamp), int(rec.timestamp), rec.zip, row.label, row.meterId, float(row.power)))
    return result

In [5]:
def getEnergy(readings):
    energy = 0
    readingCnt = len(readings)
    if readingCnt <= 1:
        return ('NONE', 0)
    for i in xrange(readingCnt-1):
        energy += (readings[i+1][0] - readings[i][0]) * readings[i][2]
    return (readings[0][1], energy)

In [ ]:
getEnergy(sorted(((1,4,3), (5,2,7), (3, 5, 9))))  # Expected to get (4, 24)

In [ ]:
test_rdd = sc.parallelize([(("a", 3), (1, 4, 3)), (("b", 1), (1, 3, 5)), (("a", 3), (5, 2, 7)), (("a", 3), (3, 5, 9))])
new_rdd = test_rdd.groupByKey().map(lambda rec: (rec[0], getEnergy(list(sorted(rec[1])))))
for row in new_rdd.collect():
    print row

In [ ]:
def writeTotalTableToCassandra(agg, tablename):
    if agg:
        cascluster = Cluster(['52.89.146.18', '52.89.197.150', '52.89.235.220', '52.89.249.32'])
        casSession = cascluster.connect('playground')
        casCommand = ('INSERT INTO %s (houseid, date, zip, label, power) VALUES ' % (tablename)) + '(%s, %s, %s, %s, %s)'
        for rec in agg:
            casSession.execute(casCommand, (str(rec[0][0]), rec[0][3], rec[0][2], rec[0][1], str(rec[1])))
        casSession.shutdown()
        cascluster.shutdown()

In [ ]:
def writePercentTableToCassandra(agg, tablename):
    if agg:
        cascluster = Cluster(['52.89.146.18', '52.89.197.150', '52.89.235.220', '52.89.249.32'])
        casSession = cascluster.connect('playground')
        casCommand = ('INSERT INTO %s (houseid, date, zip, label, percent) VALUES ' % (tablename)) + '(%s, %s, %s, %s, %s)'
        for rec in agg:
            casSession.execute(casCommand, (str(rec[0]), rec[3], rec[2], rec[1], str(rec[4])))
        casSession.shutdown()
        cascluster.shutdown()

In [6]:
def j2kwh(energy):
    return energy * 1.0 / 1000 / 3600

## Process high frequency meter readings

First of all, read in the files from HDFS

In [ ]:
ec2_host = "ec2-52-89-146-18.us-west-2.compute.amazonaws.com:9000/"
hf_hdfs_dir = "/camus/topics/testing_hf/hourly/2016/01/31/14"

highFreqDf = sqlContext.read.json("hdfs://" + ec2_host + hf_hdfs_dir)

In [ ]:
highFreqDfExp = sqlContext.createDataFrame(
    highFreqDf.flatMap(lambda row: expandRec(row)), ['houseId', 'date', 'timestamp', 'zip', 'label', 'meterId', 'power']
)

In [ ]:
highFreqMetersDf = highFreqDfExp.map(
    lambda row: ((row.houseId, row.meterId, row.zip, row.date), (row.timestamp, row.label, row.power))
).groupByKey()

In [ ]:
highFreqEnergy = highFreqMetersDf.map(lambda rec: (rec[0], getEnergy(list(sorted(rec[1])))))

In [ ]:
highFreqEnergyTot = highFreqEnergy.map(lambda rec: ((rec[0][0], rec[1][0], rec[0][2], rec[0][3]), rec[1][1])) \
                                  .reduceByKey(lambda x, y: x+y)

In [ ]:
highFreqEnergyTotNorm = highFreqEnergyTot.map(lambda rec: (rec[0], j2kwh(rec[1])))

In [ ]:
highFreqEnergyTotNorm.count()

In [ ]:
print highFreqEnergyTotNorm.take(1)

## Process Low Frequency Meter Readings

In [7]:
lf_hdfs_dir = "/camus/topics/testing_lf/hourly/2016/01/31/14"
ec2_host = "ec2-52-89-146-18.us-west-2.compute.amazonaws.com:9000/"

lowFreqDf = sqlContext.read.json("hdfs://" + ec2_host + lf_hdfs_dir)

In [ ]:
lowFreqDf.take(1)

In [ ]:
lowFreqDf.count()

In [8]:
lowFreqDfExp = sqlContext.createDataFrame(
    lowFreqDf.flatMap(lambda row: expandRec(row)), ['houseId', 'date', 'timestamp', 'zip', 'label', 'meterId', 'power']
)

In [9]:
lowFreqMetersDf = lowFreqDfExp.map (
    lambda row: ((row.houseId, row.meterId, row.zip, row.date), [(row.timestamp, row.label, row.power)])
).reduceByKey(lambda x, y: x + y) # groupByKey()

In [ ]:
lowFreqMetersDf.count()

In [ ]:
lowFreqEnergy = lowFreqMetersDf.map(lambda rec: (rec[0], getEnergy(list(sorted(rec[1])))))

In [ ]:
lowFreqEnergy.count()

In [ ]:
lowFreqEnergyTot = lowFreqEnergy.map(lambda rec: ((rec[0][0], rec[1][0], rec[0][2], rec[0][3]), rec[1][1])) \
                                  .reduceByKey(lambda x, y: x+y)

In [ ]:
lowFreqEnergyTot.count()

In [ ]:
lowFreqEnergyTotNorm = lowFreqEnergyTot.map(lambda rec: (rec[0], j2kwh(rec[1])))

In [ ]:
lowFreqEnergyTotNorm.count()

In [ ]:
print lowFreqEnergyTotNorm.take(1)

## Post-process the highFreq and lowFreq results, and write to database

In [ ]:
lfTotTable = 'batch_power_appliance'
lowFreqEnergyTotNorm.foreachPartition(lambda par: writeTotalTableToCassandra(par, lfTotTable))

In [ ]:
hfTotTable = 'batch_power_main'
highFreqEnergyTotNorm.foreachPartition(lambda par: writeTotalTableToCassandra(par, hfTotTable))

In [ ]:
def reordRec(rec):
    return ((rec[0][0], rec[0][2], rec[0][3]), (rec[0][1], rec[1]))

In [ ]:
def getPercentage(rec):
    if rec[1][1][1] < 1e-5:
        percentage = 0.0
    else:
        percentage = rec[1][0][1] * 1.0 / rec[1][1][1]
    return (rec[0][0], rec[1][0][0], rec[0][1], rec[0][2], percentage * 100)

In [ ]:
lowFreqEnergyTotNorm.take(1)

In [ ]:
highFreqEnergyTotNorm.take(1)

In [ ]:
lowFreqPercent = (lowFreqEnergyTotNorm.map(lambda rec: reordRec(rec)) \
                 .leftOuterJoin(highFreqEnergyTotNorm.map(lambda rec: reordRec(rec)))) \
                 .map(lambda rec: getPercentage(rec))

In [ ]:
lowFreqPercent.take(1)

In [ ]:
lowFreqPercent.count()

In [ ]:
percentTable = 'batch_power_percent'
lowFreqPercent.foreachPartition(lambda par: writePercentTableToCassandra(par, percentTable))